In [3]:
import sqlite3
import numpy as np
import scipy as sp
import ot
import matplotlib.pyplot as plt
import pandas as pd
import random 
import tqdm
import contextlib

In [4]:
# Let's first get our GW Distance Function
def GWDistance(A1, A2):
    C1 = sp.spatial.distance.cdist(A1, A1)
    C2 = sp.spatial.distance.cdist(A2, A2)

    C1 /= C1.max()
    C2 /= C2.max()
    p = ot.unif(len(A1))
    q = ot.unif(len(A2))

    with contextlib.redirect_stdout(None):
        gw0, log0 = ot.gromov.gromov_wasserstein(
            C1, C2, p, q, 'square_loss', verbose=True, log=True)

    return log0['gw_dist']

In [2]:
# Now let's retrieve our adjacency matrices from the various DBs
def retrieve_adjacency_matrices_from_db(db_name):
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()

    # Execute SQL commands to retrieve adjacency matrices
    cur.execute('SELECT matrix FROM adjacency_matrices')
    data = cur.fetchall()

    # Close the cursor and connection
    cur.close()
    conn.close()

    # Convert the retrieved data to adjacency matrices
    adjacency_matrices = [np.array(eval(row[0])) for row in data]

    return adjacency_matrices

In [ ]:
import random
import pandas as pd
import sqlite3

#results = []

for w1 in range(1, 7, 1):
    for w2 in range(1, 7, 1):  # Iterate over W
        for u1 in range(1, 6, 1):  # Iterate over U1
            for u2 in range(1, 6, 1):  # Iterate over U2
                results = []

                db_name1 = f'C:/Users/jmood/Desktop/Thesis/SQL_Results_50/W{w1}/W{w1}_U{u1}.db'
                db_name2 = f'C:/Users/jmood/Desktop/Thesis/SQL_Results_50/W{w2}/W{w2}_U{u2}.db'
                
                # Create a connection to the database
                conn = sqlite3.connect('C:/Users/jmood/Desktop/Thesis/GW_Distances_50_Matrices.db')

                # Create a cursor object to execute SQL queries
                cursor = conn.cursor()

                # Assuming 'your_table' is the table name in your SQL database
                cursor.execute('''
                    CREATE TABLE IF NOT EXISTS your_table (
                        Matrix1 INTEGER,
                        Matrix2 INTEGER,
                        Database1 TEXT,
                        Database2 TEXT,
                        GWDistance FLOAT,
                        Combined_Distro TEXT
                    )
                ''')

                adjacency_matrices_1 = retrieve_adjacency_matrices_from_db(db_name1)
                adjacency_matrices_2 = retrieve_adjacency_matrices_from_db(db_name2)

                #selected_matrices_1 = random.sample(adjacency_matrices_1)#, 25)
                #selected_matrices_2 = random.sample(adjacency_matrices_2)#, 25)

                for i, A1 in enumerate(adjacency_matrices_1):
                    for j, A2 in enumerate(adjacency_matrices_2):
                        gw_distance = GWDistance(A1, A2)
                        #results.append((i+1, j+1, db_name1, db_name2, gw_distance))
                        results.append((i+1, j+1, f'W{w1}_U{u1}', f'W{w2}_U{u2}', gw_distance, f'W{w1}_U{u1} W{w2}_U{u2}'))
                        print(f'Gromov-Wasserstein distance between matrix {i+1} in {db_name1} and matrix {j+1} in {db_name2}: {gw_distance}')

                # Insert the results into the database
                cursor.executemany('''
                    INSERT INTO your_table (Matrix1, Matrix2, Database1, Database2, GWDistance, Combined_Distro)
                    VALUES (?, ?, ?, ?, ?,?)
                ''', results)

                # Commit the changes and close the connection
                conn.commit()
                conn.close()
